In [1]:
import json 
from pprint import pprint

import pandas as pd

import numpy as np

In [2]:
#Load the data frame file
userDf = pd.read_pickle('userDfFile.pkl')


In [3]:
userDf.describe()

,review_count
count,1.183362e+06
mean,2.372686e+01
std,8.050645e+01
min,0.000000e+00
25%,2.000000e+00
50%,5.000000e+00
75%,1.600000e+01
max,1.165600e+04


In [4]:
userDf.dtypes

review_count      int64
user_id          object
yelping_since    object
dtype: object